In [1]:
import pandas as pd
import numpy as np
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_log_error, mean_squared_error

In [2]:
bpl.output_notebook()

Loading BokehJS ...

In [3]:
# Susceptibles equation
def dS_dt(S, I, R_t, t_inf):
    return -(R_t / t_inf) * I * S

# Expuestos equation
def dE_dt(S, E, I, R_t, t_inf, t_inc):
    return (R_t / t_inf) * I * S - (E / t_inc)

# Infectados equation
def dI_dt(I, E, t_inc, t_inf):
    return (E / t_inc) - (I / t_inf)

#Leves equation
def dL_dt(I, L, p_grave, t_inf, t_rl):
    return (1-p_grave)*(I /t_inf) - (L / t_rl)

#Graves equation
def dG_dt(I, G, p_grave, t_inf, t_hosp):
    return p_grave*(I / t_inf) - (G / t_hosp)

# Hospializados equation
def dH_dt(G, H, t_hosp, p_icu, t_rh, t_icu):
    return (G / t_hosp) - (1-p_icu)*(H / t_rh) - (p_icu)*(H / t_icu)

# ICU equation
def dICU_dt(H, ICU, p_icu, t_icu, p_m, t_ricu, t_m):
    return p_icu*(H / t_icu) - (1-p_m)*(ICU / t_ricu) - p_m*(ICU / t_m)

# Recovered equation
def dR_dt(L, H, ICU, t_rl, p_icu, t_rh, p_m, t_ricu):
    return (L / t_rl) + (1-p_icu)*(H / t_rh) + (1-p_m)*(ICU / t_ricu)

# Deaths equation
def dD_dt(ICU, t_m, p_m):
    return p_m*(ICU / t_m)


def CDMX_model(t, y, R_t, t_inf=2.9, t_inc=5.2, t_rh=12, t_rl=14, t_hosp=4, t_icu=1, t_ricu=7, t_m=8, p_m=.03, p_grave=.138, p_icu=0.05):

    if callable(R_t):
        reprod = R_t(t)
    else:
        reprod = R_t
        
    S, E, I, L, G, H, ICU, R, D = y
    
    S_out = dS_dt(S, I, reprod, t_inf)
    E_out = dE_dt(S, E, I, reprod, t_inf, t_inc)
    I_out = dI_dt(I, E, t_inc, t_inf)
    L_out = dL_dt(I, L, p_grave, t_inf, t_rl)
    G_out = dG_dt(I, G, p_grave, t_inf, t_hosp)
    H_out = dH_dt(G, H, t_hosp, p_icu, t_rh, t_icu)
    ICU_out = dICU_dt(H, ICU, p_icu, t_icu, p_m, t_ricu, t_m)
    R_out = dR_dt(L, H, ICU, t_rl, p_icu, t_rh, p_m, t_ricu)
    D_out = dD_dt(ICU, t_m, p_m)
    return [S_out, E_out, I_out, L_out, G_out, H_out, ICU_out, R_out, D_out]

In [4]:
N = 22000000
n_infected = 310*2
max_days = 200
inicio = '2020-03-15'
tiempos = pd.date_range(start=inicio, periods=max_days).values
serie_tiempos = pd.Series(tiempos)

initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0, 0, 0]

R_0 = 2.83
t_inf=2.9
t_inc=5.2
t_rh=12
t_rl=14
t_hosp=4
t_icu=1
t_ricu = 7
t_m=8
p_m=.65
p_grave=0.138
p_icu=0.05

intervenciones = [{"R":2.83,"fecha":inicio},{"R":2.2,"fecha":"2020-03-22"},{"R":0.95,"fecha":"2020-03-31"}]

def rep(t):
    rfinal = 2.83
    for intervencion in intervenciones:
        tint = serie_tiempos[(serie_tiempos-pd.Timestamp(intervencion["fecha"])).dt.days==0].index[0]
        if (t > tint):
            rfinal = intervencion["R"]
    return rfinal

args_sin_interv = (R_0, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)
args_con_interv = (rep, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)

sol_sin_interv = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args_sin_interv, t_eval=np.arange(max_days))
sol_con_interv = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args_con_interv, t_eval=np.arange(max_days))

In [5]:
labels = ["Susceptibles","Expuestos","Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]
colores = {"Susceptibles":"#969696",
           "Recuperados":"Green",
           "Expuestos":"Pink",
           "Infectados":"Red",
           "Leves":"#807DBA",
           "Graves":"#54278F",
           "Hospitalizados":"#CC4C02",
           "ICUs":"#662506",
           "Defunciones":"#525252"}

In [6]:
solucion_sin = pd.DataFrame(sol_sin_interv.y.T*N,columns=labels,index=sol_sin_interv.t)
solucion_con = pd.DataFrame(sol_con_interv.y.T*N,columns=labels,index=sol_con_interv.t)

In [7]:
solucion_sin["Fecha"] = tiempos
solucion_con["Fecha"] = tiempos

In [8]:
solucion_sin[19:28]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Fecha
19,2.196258e+07,17778.572527,6629.140101,8085.870556,749.105777,621.521363,100.526663,3419.364461,39.756526,2020-04-03
20,2.195558e+07,21033.321899,7896.425612,9590.177115,885.647900,739.602374,119.883755,4110.303255,48.709062,2020-04-04
21,2.194721e+07,24978.476204,9353.823100,11395.440292,1052.359016,877.576279,142.971023,4928.980082,59.367884,2020-04-05
22,2.193722e+07,29747.499550,11033.202936,13559.214987,1255.179658,1039.019613,170.499764,5899.040812,72.045906,2020-04-06
23,2.192544e+07,35319.722504,13065.030188,16104.772037,1491.323268,1232.254118,202.938293,7051.929871,87.148124,2020-04-07
24,2.191158e+07,41785.390523,15545.831082,19089.133204,1763.801201,1464.627116,241.094301,8423.271852,105.140687,2020-04-08
25,2.189519e+07,49383.096744,18520.595786,22613.034155,2083.466677,1741.784334,286.202805,10052.689426,126.550803,2020-04-09
26,2.187568e+07,58499.781978,21982.778409,26820.924470,2469.014776,2067.669897,339.926151,11983.803345,151.966744,2020-04-10
27,2.185239e+07,69544.319781,25947.535477,31871.765929,2939.987789,2447.989730,404.128803,14269.013170,182.093496,2020-04-11


In [9]:
solucion_con[19:28]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Fecha
19,2.198006e+07,6544.839710,3443.592230,5852.605487,494.517045,492.678576,88.583069,2988.639556,37.797063,2020-04-03
20,2.197892e+07,6436.010986,3491.423876,6446.713222,531.717794,553.313913,102.237641,3473.998495,45.539596,2020-04-04
21,2.197777e+07,6351.994312,3513.944411,7006.618672,561.457011,614.749419,117.063451,4006.539221,54.439281,2020-04-05
22,2.197663e+07,6281.152732,3523.185818,7528.064344,584.340541,675.327151,132.910890,4583.858456,64.583742,2020-04-06
23,2.197548e+07,6229.065177,3513.374481,8018.055293,603.116465,732.964552,149.591206,5202.184509,76.055773,2020-04-07
24,2.197432e+07,6190.318675,3490.107089,8476.031471,618.456917,786.870315,166.865121,5858.783513,88.913188,2020-04-08
25,2.197318e+07,6147.822624,3471.059991,8893.413427,628.885536,837.532278,184.454601,6552.027964,103.186576,2020-04-09
26,2.197205e+07,6099.092680,3458.802372,9271.202245,635.044005,884.541584,202.156927,7279.242700,118.891798,2020-04-10
27,2.197093e+07,6054.289103,3442.822846,9618.641596,639.178167,927.095916,219.774551,8036.805015,136.032435,2020-04-11


In [10]:
p = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX")

In [11]:
for label in ["ICUs"]:
    #p.line(x=solucion_sin["Fecha"],y=solucion_sin[label]/1000000,color=colores[label],line_width=3,legend_label=label,line_alpha=0.5)
    p.line(x=solucion_con["Fecha"],y=solucion_con[label]/1000,color=colores[label],line_width=3,legend_label=label,line_alpha=0.5,line_dash="dotted")

In [12]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Millones de personas'
p.legend.location = "center_right"

In [13]:
bpl.show(p)